# 7 Higher-order functions

### 7.1 Basic concepts

In [1]:
add :: Int -> Int -> Int
add x y = x + y

In [2]:
add :: Int -> (Int -> Int)
add =  \x  -> (\y -> x + y)

Line 2: Redundant lambda
Found:
add = \ x -> (\ y -> x + y)
Why not:
add x y = x + yLine 2: Avoid lambda
Found:
\ x -> (\ y -> x + y)
Why not:
(+)Line 2: Avoid lambda
Found:
(\ y -> x + y)
Why not:
(x +)

In [3]:
add 5 3

8

In [4]:
twice :: (a -> a) -> a -> a
twice f x = f (f x)

In [7]:
twice (*2) 3

12

In [8]:
twice :: (a -> a) -> (a -> a)
twice =  \f       -> \x -> f (f x)

Line 2: Redundant lambda
Found:
twice = \ f -> \ x -> f (f x)
Why not:
twice f x = f (f x)Line 2: Collapse lambdas
Found:
\ f -> \ x -> f (f x)
Why not:
\ f x -> f (f x)Line 2: Avoid lambda
Found:
\ x -> f (f x)
Why not:
f . f

In [9]:
twice reverse [1,2,3]

[1,2,3]

In [11]:
reverse [1,2,3]
reverse (reverse [1,2,3])

Line 2: Avoid reverse
Found:
reverse (reverse [1, 2, 3])
Why not:
[1, 2, 3]

[3,2,1]

[1,2,3]

In [15]:
onetwo :: (a -> a) -> (a -> a)
onetwo = \f -> \x -> f (f (f x))

Line 2: Redundant lambda
Found:
onetwo = \ f -> \ x -> f (f (f x))
Why not:
onetwo f x = f (f (f x))Line 2: Collapse lambdas
Found:
\ f -> \ x -> f (f (f x))
Why not:
\ f x -> f (f (f x))Line 2: Avoid lambda
Found:
\ x -> f (f (f x))
Why not:
f . f . f

In [13]:
onetwo reverse [1,2,3]
-- reverse (reverse (reverse [1,2,3]))

[3,2,1]

### 7.2 Processing lists

#### map

1. Defining `map` using list comprehension:  

```haskell
map :: (a -> b) -> [a] -> [b]
map f xs = [f x | x <- xs]
```

2. Defining `map` as a recursive function:  

```haskell
map :: (a -> b) -> [a] -> [b]
map f []     = []
map f (x:xs) = f x : map f xs
```

리스트 내의 모든 값들에 대하여 함수를 적용하고   
새로운 리스트를 반환해줌

In [16]:
map (+1) [1,3,5,7]

[2,4,6,8]

In [17]:
map even [1,2,3,4]

[False,True,False,True]

In [18]:
map reverse ["abc", "def", "ghi"]

["cba","fed","ihg"]

In [19]:
:type (map (+1))

(map (+1)) :: forall b. Num b => [b] -> [b]

In [27]:
map (map (+1)) [[1,2,3], [4,5]]

[[2,3,4],[5,6]]

In [28]:
map (+1) [1,2,3]
map (+1) [4,5]

[2,3,4]

[5,6]

In [29]:
[ map (+1) xs | xs <- [[1,2,3],[4,5]] ]
[ [x+1 | x <- xs] | xs <- [[1,2,3],[4,5]] ]

[[2,3,4],[5,6]]

[[2,3,4],[5,6]]

#### filter

1. Defining `filter` using list comprehension:  

```haskell
filter :: (a -> Bool) -> [a] -> [a]
filter p xs = [x | x <- xs, p x]
```

2. Defining `map` as a recursive function:  

```haskell
filter p []     = []
filter p (x:xs)
    | p x       = x : filter p xs
    | otherwise = filter p xs 
```

리스트 내의 모든 값들에 대하여 함수를 적용하고   
새로운 리스트를 반환해줌

In [30]:
filter even [1,2,3,4,5]

[2,4]

### other higher-order functions for processing lists

In [34]:
all even [0,2,4,6,8] -- 모든 원소가 조건을 만족하는지
all even [2,4,6,7,8]

True

False

In [36]:
any odd [0,2,4,6,8] -- 조건을 만족하는 원소가 한개라도 존재하는지
any odd [2,4,6,7,8]

False

True

In [41]:
take 3 [1,2,3,4,5] -- 고차함수 아님
drop 3 [1,2,3,4,5] -- 고차함수 아님

[1,2,3]

[4,5]

In [40]:
takeWhile even [2,4,6,7,8,9] -- 7부터는 조건이 만족안해서 While 종료

[2,4,6]

In [39]:
dropWhile odd [1,3,5,6,7]

[6,7]

In [45]:
-- 문자열에서 첫번째 단어만을 뽑은 문자열을 계산하는 함수
import Data.Char (isSpace)
takeWhile (not . isSpace) (dropWhile isSpace "  hello  world nice")

firstWord =  takeWhile (not . isSpace) . dropWhile isSpace
firstWord "  hello  world nice"

"hello"

"hello"

In [ ]:
### 7.3 The `foldr` function

A common simple pattern of recursion over lists:  
```haskell
f []     = v
f (x:xs) = x ☆ f xs
```

The `foldr` function encapsulates this pattern of recursion:
```haskell
f = foldr (☆) v
```

The behavior of `foldr` can be summarized as follows:
```haskell
foldr (☆) v [x0,x1,.. ..,xn]
 == x0 ☆ (x1 ☆ ( .. ..  (xn ☆ v) .. ..))
```
The name fold right reflects the use of an operator that is assumed to associate to the right.

<hr>

Let us revisit the common simple pattern of recursion over lists:
```haskell
f []     = v
f (x:xs) = x ☆ f xs
```

In [59]:
foldr' :: (a -> b -> b) -> b -> [a] -> b
foldr' (☆) v []  = v
foldr' (☆) v (x:xs) = x ☆ (foldr (☆) v xs)

Line 3: Redundant bracket
Found:
x ☆ (foldr (☆) v xs)
Why not:
x ☆ foldr (☆) v xs

In [60]:
-- v = 0         (☆) = (+)
sum []     = 0
sum (x:xs) = x + sum xs
-- v = 1         (☆) = (*)
product []     = 1
product (x:xs) = x * product xs
-- v = False     (☆) = (||)
or []     = False
or (x:xs) = x || or xs
-- v = True      (☆) = (&&)
and []    = True
and (x:xs) = x && and xs

Line 2: Use foldr
Found:
sum [] = 0
sum (x : xs) = x + sum xs
Why not:
sum xs = foldr (+) 0 xsLine 5: Use foldr
Found:
product [] = 1
product (x : xs) = x * product xs
Why not:
product xs = foldr (*) 1 xsLine 8: Use foldr
Found:
or [] = False
or (x : xs) = x || or xs
Why not:
or xs = foldr (||) False xsLine 11: Use foldr
Found:
and [] = True
and (x : xs) = x && and xs
Why not:
and xs = foldr (&&) True xs

In [61]:
sum     = foldr (+)  0
product = foldr (*)  1
or      = foldr (||) False
and     = foldr (&&) True

In [63]:
-- 똑같은 리스트를 떼어냈다가 새로 만들어내는 함수
foldr' (:) [] [1,2,3,4] -- 즉 List가 foldr 구조로 이루어짐

[1,2,3,4]

In [65]:
copylist = foldr' (:) []
copylist [1,2,3,4]

[1,2,3,4]

In [64]:
[1,2,3,4] == 1 : (2 : (3 : (4 : [])))

Line 1: Use list literal
Found:
1 : (2 : (3 : (4 : [])))
Why not:
[1, 2, 3, 4]

True

In [66]:
concat [ [1,2], [3,4], [5], [6,7,8] ]

[1,2,3,4,5,6,7,8]

In [68]:
concat xss = [x | xs <- xss, x <- xs]

In [69]:
concat [ [1,2], [3,4] ]

Line 1: Use ++
Found:
concat [[1, 2], [3, 4]]
Why not:
[1, 2] ++ [3, 4]

[1,2,3,4]

In [74]:
concat :: [[a]] -> [a]
concat []       = []
concat (xs:xss) = xs ++ concat xss  

Line 2: Use foldr
Found:
concat [] = []
concat (xs : xss) = xs ++ concat xss
Why not:
concat xss = foldr (++) [] xss

In [75]:
concat [ [1,2,3],[4,5] ]

Line 1: Use ++
Found:
concat [[1, 2, 3], [4, 5]]
Why not:
[1, 2, 3] ++ [4, 5]

[1,2,3,4,5]

In [76]:
concat = foldr (++) []

In [77]:
concat [ [1,2,3],[4,5] ]

Line 1: Use ++
Found:
concat [[1, 2, 3], [4, 5]]
Why not:
[1, 2, 3] ++ [4, 5]

[1,2,3,4,5]

In [78]:
foldl :: (b -> a -> b) -> b -> [a] -> b
foldl (☆) v []     = v
foldl (☆) v (x:xs) = foldl (☆) (v ☆ x) xs

In [79]:
-- ((((100 -1) - 2) - 3) - 4) - 5
foldl (-) 100 [1,2,3,4,5]

85

### 7.5 The composition operator
- $f ∘ g$ in mathematics
- `f . g` in Haskell

```haskell
(.) :: (b -> c) -> (a -> b) -> (a -> c)
f . g = \x -> f (g x)
```

함수 2개를 받아서 함수 한개를 만들어냄  
(b -> c) == f  
(a -> b) == g  

The identity function(항등 함수) `id = \x -> x` is the identity for the composition operator `(.)`. That is,
```haskell
id . f == f == f . id
```

항등원
- 항등원 연산자 x == x
- x 연산자 항등원 == x
- \+ 항등원은 0
- \* 항등원은 1

In [84]:
compose :: [(a -> a)] -> (a -> a)
compose = foldr (.) id

Line 1: Redundant bracket
Found:
[(a -> a)]
Why not:
[a -> a]

In [85]:
compose [(*4),(+1),(^2)] 2

20

### 7.6 Binary string transmitter
### 7.7 Voting algorithms